# Script saldo Cajamar, OpenBank y Unicaja

## Requisites

In [0]:
!pip install plotly

## Code

### Imports

In [18]:
import plotly as py
import plotly.graph_objs as go

import numpy as np
import datetime as dt
import os.path
import xlrd

py.offline.init_notebook_mode(connected=True)

### Read CSV

In [11]:
def processCSV(files, splitToken, bank):
    fecha = []
    saldo = []
    
    if bank == 'cajamar':
        for file in files:
            first = True
            if os.path.exists(file):
                with open(file) as f:
                    for line in f:
                        if first is False:
                            tokens = line.split(splitToken)
                            fecha.append(dt.datetime.strptime(tokens[0], '%d/%m/%Y').date())
                            saldo.append(float(tokens[len(tokens)-1][:-1].replace('.', '').replace(',', '.')))
                        first = False

    if bank == 'openbank':
        for file in files:
            first = 0
            if os.path.exists(file):
                with open(file) as f:
                    for line in f:                        
                        if first > 11:
                            tokens = line.split(splitToken)
                            fecha.append(dt.datetime.strptime(tokens[1], '%d/%m/%Y').date())
                            saldo.append(float(tokens[len(tokens)-1][:-1].replace('.', '').replace(',', '.')))
                        first += 1

    fecha = np.array(fecha)
    saldo = np.array(saldo)

    return (saldo, fecha)

### Read XLS

In [9]:
def processXLS(file, bank):   
    balance = []
    date = []
    if bank == 'unicaja':
        book = xlrd.open_workbook(file)
        first_sheet = book.sheet_by_index(0)
        date_int = first_sheet.col_values(0)[5:]
        balance = first_sheet.col_values(5)[5:]
        date = [xlrd.xldate_as_datetime(date_int[i], book.datemode).date().isoformat() for i in  xrange(5, len(date_int))]
        
    return (np.array(balance), np.array(date))     

### Process data files

#### Unicaja

In [ ]:
file = "accounts/current/unicaja/unicaja.xls"
(saldosUnicaja, fechasUnicaja) = processXLS(file, 'unicaja')

#### Cajamar y Openbank

In [12]:
files = ['saldo.csv']
(saldosCajamar, fechasCajamar) = processCSV(files, '\t', 'cajamar')

files = ['Nomina.csv']
(saldosOpenNomina, fechasOpenNomina) = processCSV(files, '\t', 'openbank')

files = ['Ahorro.csv']
(saldosOpenAhorro, fechasOpenAhorro) = processCSV(files, '\t', 'openbank')

saldosOpenAhorro = np.insert(saldosOpenAhorro, 0, saldosOpenAhorro[0], axis=0)
fechasOpenAhorro = np.insert(fechasOpenAhorro, 0, fechasOpenNomina[0], axis=0)

IndexError: index 0 is out of bounds for axis 0 with size 0

### Traces

#### Unicaja

In [14]:
trace_main_unicaja = go.Scatter(
    x = fechasUnicaja,
    y = saldosUnicaja,
    name = 'Unicaja: Saldo ' + str(format(saldosUnicaja[-1], ',.2f').replace(",", "X").replace(".", ",").replace("X", ".")) + '€',
    line = dict(
        color = 'green'
    ),
    mode = 'lines'
)

data = [trace_main_unicaja]

#### Cajamar y Openbank

In [59]:
trace_main_cajamar = go.Scatter(
    x = fechasCajamar,
    y = saldosCajamar,
    name = 'Cajamar: Saldo ' + str(format(saldosCajamar[-1], ',.2f').replace(",", "X").replace(".", ",").replace("X", ".")) + '€',
    line = dict(
        color = 'green'
    ),
    mode = 'lines'
)
trace_max_cajamar = go.Scatter(
    x = fechasCajamar,
    y = [saldosCajamar.max() for f in fechasCajamar],
    name = 'Cajamar: Saldo máximo',
    #visible = 'legendonly',
    #hoverinfo = 'name',
    line = dict(
        color = 'cyan',
        width = 4,
        dash = 'dot'
    )
)
trace_mean_cajamar = go.Scatter(
    x = fechasCajamar,
    y = [saldosCajamar.mean() for f in fechasCajamar],
    name = 'Cajamar: Saldo medio',
    #hoverinfo = 'none',
    line = dict(
        color = 'magenta',
        width = 4,
        dash = 'dashdot'
    )
)
trace_min_cajamar = go.Scatter(
    x = fechasCajamar,
    y = [saldosCajamar.min() for f in fechasCajamar],
    name = 'Cajamar: Saldo mínimo',
    line = dict(
        color = 'red',
        width = 4,
        dash = 'dot'
    )
)

trace_main_openAhorro = go.Scatter(
    x = fechasOpenAhorro,
    y = saldosOpenAhorro,
    name = 'OpenAhorro: Saldo ' + str(format(saldosOpenAhorro[0], ',.2f').replace(",", "X").replace(".", ",").replace("X", ".")) + '€',
    line = dict(
        color = 'blue'
    ),
    mode = 'lines'
)

trace_main_openNomina = go.Scatter(
    x = fechasOpenNomina,
    y = saldosOpenNomina,
    name = 'OpenNomina: Saldo ' + str(format(saldosOpenNomina[0], ',.2f').replace(",", "X").replace(".", ",").replace("X", ".")) + '€',
    line = dict(
        color = 'darkred'
    ),
    mode = 'lines'
)

data = [trace_main_cajamar, trace_max_cajamar, trace_mean_cajamar, trace_min_cajamar,
        trace_main_openAhorro, trace_main_openNomina]

### Plot

#### Unicaja

In [16]:
layout = go.Layout(title = 'Saldo ' + str(fechasUnicaja[-1]),
              xaxis = dict(title = 'Fecha'),
              yaxis = dict(title = 'Saldo (€)'),
              showlegend = True
)

fig = dict(data=data, layout=layout)
py.offline.iplot(fig, filename='styled-line')

#### Cajamar y Openbank

In [15]:
layout = go.Layout(title = 'Saldo ' + str(fechasCaja[-1]) + ' / ' + str(fechasOpen[0]),
              xaxis = dict(title = 'Fecha'),
              yaxis = dict(title = 'Saldo (€)'),
              showlegend = True
)

fig = dict(data=data, layout=layout)
py.offline.iplot(fig, filename='styled-line')

NameError: name 'fechasCaja' is not defined